In [1]:
import requests
from bs4 import BeautifulSoup
import re
import warnings
import json
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

# GET LINKs

In [2]:
path = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
r = requests.get(f'{path}')
soup = BeautifulSoup(r.content, 'html.parser')
links = []
base = 'https://www.imdb.com'

In [5]:
path = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
r = requests.get(f'{path}')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('tbody', class_ = 'lister-list')
movies = s.find_all('td',class_ = 'titleColumn')
links = []
base = 'https://www.imdb.com'
for movie in movies:
    link = movie.a['href']
    links.append(f'{link}')

save links

In [7]:
links = np.array(links)
np.save('top_250_Linkes',links)

load links

In [10]:
links = np.load('top_250_Linkes.npy')

# GET DATA

In [33]:
headers = {'User-Agent': 'Safari/605.1.15'}
xrror = []

movies = pd.DataFrame(columns = ['id','title','year','runtime','parental_guide','gross_us_canda'])
genre_movie =  pd.DataFrame(columns = ['movie_id','genre'])
cast = pd.DataFrame(columns = ['movie_id','person_id'])
crew = pd.DataFrame(columns = ['movie_id','person_id','role'])
person = pd.DataFrame(columns = ['person_id','name'])

In [34]:
for link in links[0:25]:
    r = requests.get(f'{base}{link}', headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        movie_id = link[9:-1]
        title = soup.find('span', class_ = 'sc-afe43def-1 fDTGTb').text
        info = soup.find_all('li',role ='presentation',class_="ipc-inline-list__item")
        year = int(info[3].text)
        parental_guide = info[4].text
        runtime = info[5].text
        # TODO to min 
        gross_us_canda = soup.find_all('span', class_ = 'ipc-metadata-list-item__list-content-item')[3].text
        movies.loc[len(movies.index)] = [movie_id,title,year,runtime,parental_guide,gross_us_canda]
        genres = soup.find_all('a', class_ = 'ipc-chip ipc-chip--on-baseAlt')
        for g in genres:
            genre_movie.loc[len(genre_movie.index)] = [movie_id,g.text]
            
        for i in soup.find_all('li', 
            class_ = 'ipc-metadata-list__item')[:3]:
            span = i.find_all('a'
                ,class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
            role = f'{i.text[:8]}'
    #         print(role+'\n')
            for s in span:
    #             print(s.text)
                name = s.text
    #             print(s['href'],'h')
                p_link = s['href']
                person.loc[len(person.index)] = [p_link[8:-15],name]
                person.drop_duplicates(inplace = True)
                if (role[:5] == 'Stars'):
                    cast.loc[len(cast.index)] = [movie_id,p_link[8:-15]]
                else:
                    crew.loc[len(crew.index)] = [movie_id,p_link[8:-15],role]
    except Exception as e: 
        print(e)       
        xrror.append(link)

In [35]:
cast

,movie_id,person_id
0,0111161,0000209
1,0111161,0000151
2,0111161,0348409
3,0068646,0000008
4,0068646,0000199
...,...,...
70,0317248,1129884
71,0317248,0618690
72,0816692,0000190
73,0816692,0004266


save links[0:40] data (without cast and crew)

In [36]:
cast.to_csv('cast.csv',index=False)
crew.to_csv('crew.csv',index=False)
person.to_csv('person.csv',index=False)
genre_movie.to_csv('genre_movie.csv',index=False)
movies.to_csv('movies.csv',index=False)